IBM- PNT2022TMID05694- Deep Learning Fundus Image Analysis For Early Detection Of Diabetic Retinopathy

In [ ]:
from google.colab import drive
drive.mount('/Drive')

Mounted at /Drive


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imageSize = [299, 299]

trainPath = r"/Drive/MyDrive/ibm/preprocessed dataset/preprocessed dataset/training"

testPath = r"/Drive/MyDrive/ibm/preprocessed dataset/preprocessed dataset/testing"

In [ ]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 4s 0us/step


In [ ]:
for layer in xception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/Drive/MyDrive/ibm/preprocessed dataset/preprocessed dataset/training',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/Drive/MyDrive/ibm/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3662 images belonging to 5 classes.
Found 590 images belonging to 5 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set)//32,
  validation_steps=len(test_set)//32
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/30
3/3 [==============================] - 117s 40s/step - loss: 9.3656 - accuracy: 0.3646
Epoch 2/30
3/3 [==============================] - 78s 26s/step - loss: 10.3674 - accuracy: 0.5104
Epoch 3/30
3/3 [==============================] - 75s 25s/step - loss: 9.8752 - accuracy: 0.5729
Epoch 4/30
3/3 [==============================] - 77s 26s/step - loss: 5.8951 - accuracy: 0.5417
Epoch 5/30
3/3 [==============================] - 75s 25s/step - loss: 5.2364 - accuracy: 0.5833
Epoch 6/30
3/3 [==============================] - 75s 25s/step - loss: 4.1381 - accuracy: 0.6771
Epoch 7/30
3/3 [==============================] - 65s 21s/step - loss: 3.6492 - accuracy: 0.7396
Epoch 8/30
3/3 [==============================] - 66s 21s/step - loss: 3.1862 - accuracy: 0.6771
Epoch 9/30
3/3 [==============================] - 63s 22s/step - loss: 3.3781 - accuracy: 0.6354
Epoch 10/30
3/3 [==============================] - 68s 24s/step - loss: 2.3233 - accuracy: 0.7500
Epoch 11/30
3/3 [==========

In [ ]:
model.save('Updated-Xception-diabetic-retinopathy.h5')

In [ ]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [ ]:
model=load_model('updated-xception-diabetic-retinopathy.h5')

In [ ]:
img=image.load_img(r'/Drive/MyDrive/ibm/preprocessed dataset/preprocessed dataset/testing/0/cfb17a7cc8d4.png',target_size=(299,299))

In [ ]:
x=image.img_to_array(img)

In [ ]:
import matplotlib.pyplot as plt
x.shape

(299, 299, 3)

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 299, 299, 3)

In [ ]:
model.predict(img_data)

1/1 [==============================] - 1s 1s/step


array([[1.0000000e+00, 1.3992481e-20, 2.4709993e-12, 1.2183851e-20,
        9.3386634e-19]], dtype=float32)

In [ ]:
output=np.argmax(model.predict(img_data), axis=1)

1/1 [==============================] - 0s 22ms/step


In [ ]:
output==0,output==1,output==2,output==3,output==4

(array([ True]),
 array([False]),
 array([False]),
 array([False]),
 array([False]))

In [ ]:
index=['No Diabetic Retinopathy', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR']
result = str(index[output[0]])
result

'No Diabetic Retinopathy'